In [1]:
## TASK 2
import pandas as pd
from glob import glob

DATA_PATH1 = "data/DayAheadPrices_12.1.D/" # different price zones in Europe
DATA_PATH2 = "data/PhysicalFlows_12.1.G/" # hourly transmission flows of electricity between price zones

# Read the datapaths
df1 = pd.concat([pd.read_csv(file, sep='\t', parse_dates=['DateTime']) for file in glob(f"{DATA_PATH1}/*.csv")])
df2 = pd.concat([pd.read_csv(file, sep='\t', parse_dates=['DateTime']) for file in glob(f"{DATA_PATH2}/*.csv")])

# Select relevant columns for each datafile
df1 = df1[['DateTime', 'ResolutionCode', 'MapCode', 'Price']]
df2 = df2[['DateTime', 'InMapCode', 'OutMapCode', 'FlowValue']]

# Select the right price zones (NO2 and DE_LU) using built-in function isin
filtered_prices_df = df1[df1['MapCode'].isin(['NO2', 'DE_LU'])] 
#filtered_prices_df = filtered_prices_df[filtered_prices_df['ResolutionCode'] == PT60M]  ?????

filtered_flows_df = df2[(df2['InMapCode'].isin(['NO2', 'DE_LU'])) & (df2['OutMapCode'].isin(['NO2', 'DE_LU']))]

# Combine the datafiles based on the selected columns 
df = pd.merge(filtered_prices_df, filtered_flows_df, on='DateTime', how='inner')

In [2]:
df

,DateTime,ResolutionCode,MapCode,Price,InMapCode,OutMapCode,FlowValue
0,2021-05-12 22:00:00,PT60M,DE_LU,65.04,DE_LU,NO2,1204.04
1,2021-05-12 22:00:00,PT60M,DE_LU,65.04,NO2,DE_LU,0.00
2,2021-05-12 23:00:00,PT60M,DE_LU,63.15,DE_LU,NO2,1203.96
3,2021-05-12 23:00:00,PT60M,DE_LU,63.15,NO2,DE_LU,0.00
4,2021-05-13 00:00:00,PT60M,DE_LU,60.74,DE_LU,NO2,1204.08
...,...,...,...,...,...,...,...
153267,2021-04-30 22:00:00,PT60M,DE_LU,69.29,NO2,DE_LU,0.00
153268,2021-04-30 23:00:00,PT60M,NO2,50.79,DE_LU,NO2,1264.64
153269,2021-04-30 23:00:00,PT60M,NO2,50.79,NO2,DE_LU,0.00
153270,2021-04-30 23:00:00,PT60M,DE_LU,64.60,DE_LU,NO2,1264.64


 
Your boss has heard that electricity prices have become more volatile with extreme prices in 
recent years. Therefore, she has asked you to create a table with some descriptive statistics for 
the hourly electricity price in NO2 and Germany. The table should: 
- contain the mean, median, standard deviation, min and max of the hourly electricity 
price in NO2 and Germany separately for each year in the sample (2019, 2020, 2021, 
2022, 2023). 
- round all descriptive statistics to two decimals. 
- be stored as an excel file called “table_task2.xlsx”. 
What is your conclusion? Have electricity prices in NO2 and Germany become more extreme in 
recent years? Have the prices evolved differently in NO2 than in Germany?  

In [37]:
# Filter for hourly resolution only
filtered_prices_df = filtered_prices_df[filtered_prices_df['ResolutionCode'] == 'PT60M']

# Seperate the prices for Norway(NO2) and Germany(DE_LU)
price_data_NO = filtered_prices_df[filtered_prices_df['MapCode'].isin(['NO2'])]
Price_data_DE = filtered_prices_df[filtered_prices_df['MapCode'].isin(['DE_LU'])]

# Calculate the required statistics (mean, median, std, min, and max)
statistics_NO_yearly = price_data_NO[['Price']].resample('YE').agg(['mean', 'median', 'std', 'min', 'max']).round(2)
statistics_DE_yearly = Price_data_DE[['Price']].resample('YE').agg(['mean', 'median', 'std', 'min', 'max']).round(2)

# Merge the tables 
statistics = pd.merge(statistics_NO_yearly, statistics_DE_yearly, left_on="DateTime", right_on='DateTime', suffixes=('_NORWAY', '_GERMANY'))
display(statistics)

# Store the table as an Excel file
statistics.to_excel("table_task2.xlsx")

Price_NORWAY                                Price_GERMANY          \
                   mean  median     std    min     max          mean  median   
DateTime                                                                       
2019-12-31        39.26   38.81    8.23   5.86  109.45         37.67   38.06   
2020-12-31         9.29    6.95    8.26  -1.73   99.92         30.47   30.99   
2021-12-31        75.11   59.34   47.22  -1.97  600.16         96.85   75.48   
2022-12-31       211.28  181.13  125.85   0.04  844.00        235.44  208.34   
2023-12-31        79.44   80.46   36.28 -61.84  261.85         95.18   98.02   

                                    
               std     min     max  
DateTime                            
2019-12-31   15.52  -90.01  121.46  
2020-12-31   17.50  -83.94  200.04  
2021-12-31   73.68  -69.00  620.00  
2022-12-31  142.82  -19.04  871.00  
2023-12-31   47.58 -500.00  524.27